In [ ]:
# 0.0 import packages

import sys
import string
import mido
import numpy as np
import pandas as pd

import audiosegment
import librosa
import torch
import shutil
import os

import matplotlib.pyplot as plot

In [ ]:
# 0.2 GPU stuff

print ("cuda: ", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print ("current device: ", device)
print ("count: ", torch.cuda.device_count())

if torch.cuda.is_available():
    print ("device name: ", torch.cuda.get_device_name(0))
    torch.cuda.set_device(0)

In [ ]:
# 1.0 load in datasets

train_feats = np.load("data/train_feats.npy", allow_pickle=True)
train_labels = np.load("data/train_labels.npy", allow_pickle=True)
test_feats = np.load("data/test_feats.npy", allow_pickle=True)
test_labels = np.load("data/test_labels.npy", allow_pickle=True)
val_feats = np.load("data/val_feats.npy", allow_pickle=True)
val_labels = np.load("data/val_labels.npy", allow_pickle=True)

print ("train feats: ", len(train_feats), " type: ", type(train_feats))
print ("train labels: ", len(train_labels), " type: ", type(train_labels))

print ("test feats: ", len(test_feats), " type: ", type(test_feats))
print ("test labels: ", len(test_labels), " type: ", type(test_labels))

print ("val feats: ", len(val_feats), " type: ", type(val_feats))
print ("val labels: ", len(val_labels), " type: ", type(val_labels))

# create tensors
train_feats_tensor = torch.tensor(train_feats, requires_grad=True)
train_labels_tensor = torch.tensor(train_labels)

test_feats_tensor = torch.tensor(test_feats, requires_grad=True)
test_labels_tensor = torch.tensor(test_labels)

val_feats_tensor = torch.tensor(val_feats, requires_grad=True)
val_labels_tensor = torch.tensor(val_labels)